In [12]:
# public libraries
import pandas as pd
import numpy as np
import pickle as pkl

## compiling the datasets

In [2]:
# ERGO-II

# mcpas test 
results = pkl.load(open('../external/ERGO-II/Samples/mcpas_test_samples.pickle', 'rb'))
results = pd.DataFrame(results)[['tcrb','peptide','sign']].dropna()
results = results.loc[(results != 'UNK').all(1)].reset_index().iloc[:, 1:]
results.columns = ['CDR3','Epitope','y']
results.to_csv('../outputs/model_comparison/external.ergoii.mcpas_test.csv')
# vdjdb test (w/o 10x)
results = pkl.load(open('../external/ERGO-II/Samples/vdjdb_no10x_test_samples.pickle', 'rb'))
results = pd.DataFrame(results)[['tcrb','peptide','sign']].dropna()
results = results.loc[(results != 'UNK').all(1)].reset_index().iloc[:, 1:]
results.columns = ['CDR3','Epitope','y']
results.to_csv('../outputs/model_comparison/external.ergoii.vdjdb_no10x_test.csv')
# vdjdb test 
results = pkl.load(open('../external/ERGO-II/Samples/vdjdb_test_samples.pickle', 'rb'))
results = pd.DataFrame(results)[['tcrb','peptide','sign']].dropna()
results = results.loc[(results != 'UNK').all(1)].reset_index().iloc[:, 1:]
results.columns = ['CDR3','Epitope','y']
results.to_csv('../outputs/model_comparison/external.ergoii.vdjdb_test.csv')

In [3]:
# Net-TCR-2.0

# mira 90
results = pd.read_csv('../external/NetTCR-2.0/data/mira_eval_threshold90.csv')
results.columns = ['CDR3','Epitope','y']
results.to_csv('../outputs/model_comparison/external.nettcr2.mira_eval_threshold90.csv')

# mira 92
results = pd.read_csv('../external/NetTCR-2.0/data/mira_eval_threshold92.csv')
results.columns = ['CDR3','Epitope','y']
results.to_csv('../outputs/model_comparison/external.nettcr2.mira_eval_threshold92.csv')

# mira 94
results = pd.read_csv('../external/NetTCR-2.0/data/mira_eval_threshold94.csv')
results.columns = ['CDR3','Epitope','y']
results.to_csv('../outputs/model_comparison/external.nettcr2.mira_eval_threshold94.csv')

# mira 99
results = pd.read_csv('../external/NetTCR-2.0/data/mira_eval_threshold99.csv')
results.columns = ['CDR3','Epitope','y']
results.to_csv('../outputs/model_comparison/external.nettcr2.mira_eval_threshold99.csv')

In [4]:
# pMT-net (could not find binder column in https://github.com/tianshilu/pMTnet/blob/master/data/testing_data.csv)
# ^ in most recent commit 4cb60c4c14fe90e3107ba82fb4abaa250bd70654

In [5]:
# ImRex (could not finder testing data formatted in https://github.com/pmoris/ImRex/tree/master/data)
# ^ in most recent commit ec52762a5f4ca3ef86e2f088fa90fd884b64b1c5

In [16]:
# ATM-TCR

# covid-19
results = pd.read_csv('../external/ATM-TCR/data/covid19_data.csv', index_col=None, header=None)
results.columns = ['Epitope','CDR3','y']
results[['CDR3','Epitope','y']].to_csv('../outputs/model_comparison/external.atmtcr.covid19.csv')

# combined is of bad quality with non-traditional or aberrant amino acids such as O

In [7]:
# AVIB (the test datasets will be separately written out because it requires on the fly random splitting)

## converting them to readable formats by each model

In [14]:
from glob import glob

# we want to test on all on the externals
for fn_org in glob('../outputs/model_comparison/external.*.csv'):
    # read in the external data
    df_ext = pd.read_csv(fn_org, index_col=0).reset_index().iloc[:, 1:]
    X = df_ext[['CDR3','Epitope']]
    y = df_ext['y']
    
    # write for TARPON
    fn = fn_org.replace('.csv','.TARPON.csv').replace('external.','external_test.')
    X.to_csv(fn)
    
    # write for ATM-TCR
    fn = fn_org.replace('.csv','.ATM-TCR.csv').replace('external.','external_test.')
    pd.concat([X, y], axis=1).to_csv(fn, header=False, index=False)

    # write for pMTnet
    tmp = X.copy()
    tmp['HLA'] = 'A*02:01'
    tmp.columns = ['CDR3','Antigen','HLA']
    fn = fn_org.replace('.csv','.pMTnet.csv').replace('external.','external_test.')
    tmp.to_csv(fn, index=False)

#     # ignored because only restricted to three antigens
#     # write for NetTCR2
#     fn = fn_org.replace('.csv','.NetTCR2.csv').replace('external.','external_test.')
#     X['CDR3'].to_csv(fn, header=False, index=False)

    # write for NetTCR2 (CLI version)
    fn = fn_org.replace('.csv','.NetTCR2_CLI.csv').replace('external.','external_test.')
    tmp = X.copy()
    tmp.columns = ['CDR3b','peptide']
    tmp['CDR3a'] = None
    tmp[['CDR3a','CDR3b','peptide']].to_csv(fn, index=False)

    # write for ImRex
    fn = fn_org.replace('.csv','.ImRex.csv').replace('external.','external_test.')
    tmp = X.copy()
    tmp.columns = ['cdr3','antigen.epitope']
    tmp.to_csv(fn, index=False, sep=';')

    # write for ERGO-II
    fn = fn_org.replace('.csv','.ERGOII.csv').replace('external.','external_test.')
    tmp = X.copy()
    X.columns = ['TRB','Peptide']
    X['MHC'] = 'HLA-A*02'
    X['T-Cell-Type'] = 'CD8'
    X = X.T.reindex(['TRA','TRB','TRAV','TRAJ','TRBV','TRBJ','T-Cell-Type','Peptide','MHC']).T
    X.to_csv(fn, index=False)

## testing them on each model

In [24]:
# # NetTCR-2.0

# # on NetTCR-2.0
# python nettcr.py -tr test/sample_train.csv -c b \
# -te ../../outputs/model_comparison/external_test.nettcr2.mira_eval_threshold90.NetTCR2_CLI.csv \
# -o ../../outputs/model_comparison/external_test_nettcr2.nettcr2.mira_eval_threshold90.csv

In [ ]:
# # ERGO-II (v2 is simply Predict.py but writing it to sys.argv[3])

# # on NetTCR-2.0
# python Predictv2.py vdjdb ../../outputs/model_comparison/external_test.nettcr2.mira_eval_threshold90.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiivdjdb.nettcr2.mira_eval_threshold90.csv
# python Predictv2.py mcpas ../../outputs/model_comparison/external_test.nettcr2.mira_eval_threshold90.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiimcpas.nettcr2.mira_eval_threshold90.csv

# # on ERGO-II
# python Predictv2.py vdjdb ../../outputs/model_comparison/external_test.ergoii.mcpas_test.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiivdjdb.ergoii.mcpas_test.csv
# python Predictv2.py mcpas ../../outputs/model_comparison/external_test.ergoii.mcpas_test.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiimcpas.ergoii.mcpas_test.csv

# # on ATM-TCR (fails on combined)
# python Predictv2.py vdjdb ../../outputs/model_comparison/external_test.atmtcr.combined.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiivdjdb.atmtcr.combined.csv
# python Predictv2.py mcpas ../../outputs/model_comparison/external_test.atmtcr.combined.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiimcpas.atmtcr.combined.csv

# # on AVIB
# python Predictv2.py vdjdb ../../outputs/model_comparison/external_test.avib.alpha_beta_splits.beta.rep_0.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiivdjdb.avib.alpha_beta_splits.beta.rep_0.csv
# python Predictv2.py mcpas ../../outputs/model_comparison/external_test.avib.alpha_beta_splits.beta.rep_0.ERGOII.csv \
# ../../outputs/model_comparison/external_test_ergoiimcpas.avib.alpha_beta_splits.beta.rep_0.csv


In [ ]:
# ImRex

# # on NetTCR-2.0
# python ./src/scripts/predict/predict.py --model ./models/pretrained/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001.h5 \
# --input ../../outputs/model_comparison/external_test.nettcr2.mira_eval_threshold90.ImRex.csv \
# --output ../../outputs/model_comparison/external_test_imrex.nettcr2.mira_eval_threshold90.csv

# # on ERGO-II
# python ./src/scripts/predict/predict.py --model ./models/pretrained/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001.h5 \
# --input ../../outputs/model_comparison/external_test.ergoii.mcpas_test.ImRex.csv \
# --output ../../outputs/model_comparison/external_test_imrex.ergoii.mcpas_test.csv

# # on ATM-TCR
# python ./src/scripts/predict/predict.py --model ./models/pretrained/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001.h5 \
# --input ../../outputs/model_comparison/external_test.atmtcr.combined.ImRex.csv \
# --output ../../outputs/model_comparison/external_test_imrex.atmtcr.combined.csv

# # on AVIB
# python ./src/scripts/predict/predict.py --model ./models/pretrained/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001.h5 \
# --input ../../outputs/model_comparison/external_test.avib.alpha_beta_splits.beta.rep_0.ImRex.csv \
# --output ../../outputs/model_comparison/external_test_imrex.avib.alpha_beta_splits.beta.rep_0.csv

In [ ]:
# pMTnet

# # on NetTCR-2.0
# python pMTnet.py \
# -input /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.nettcr2.mira_eval_threshold90.pMTnet.csv \
# -library /ssd1/dchen/GNN_PEPTIDE_TCR/external/pMTnet/library \
# -output /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_pmtnet.nettcr2.mira_eval_threshold90 \
# -output_log /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/TMP.log

# # on ERGO-II
# python pMTnet.py \
# -input /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.ergoii.mcpas_test.pMTnet.csv \
# -library /ssd1/dchen/GNN_PEPTIDE_TCR/external/pMTnet/library \
# -output /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_pmtnet.ergoii.mcpas_test \
# -output_log /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/TMP.log

# # on ATM-TCR
# python pMTnet.py \
# -input /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.atmtcr.combined.pMTnet.csv \
# -library /ssd1/dchen/GNN_PEPTIDE_TCR/external/pMTnet/library \
# -output /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_pmtnet.atmtcr.combined \
# -output_log /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/TMP.log

# # on AVIB
# python pMTnet.py \
# -input /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.avib.alpha_beta_splits.beta.rep_0.pMTnet.csv \
# -library /ssd1/dchen/GNN_PEPTIDE_TCR/external/pMTnet/library \
# -output /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_pmtnet.avib.alpha_beta_splits.beta.rep_0 \
# -output_log /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/TMP.log

In [ ]:
# ATM-TCR

# # on NetTCR-2.0
# python main.py --infile /ssd1/dchen/GNN_PEPTIDE_TCR/external/ATM-TCR/data/combined_dataset.csv \
# --indepfile /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.nettcr2.mira_eval_threshold90.ATM-TCR.csv \
# --model_name original.ckpt --mode test --save_model False --cuda False

# # on ERGO-II
# python main.py --infile /ssd1/dchen/GNN_PEPTIDE_TCR/external/ATM-TCR/data/combined_dataset.csv \
# --indepfile /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.ergoii.mcpas_test.ATM-TCR.csv \
# --model_name original.ckpt --mode test --save_model False --cuda False

# # on ATM-TCR
# python main.py --infile /ssd1/dchen/GNN_PEPTIDE_TCR/external/ATM-TCR/data/combined_dataset.csv \
# --indepfile /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.atmtcr.combined.ATM-TCR.csv \
# --model_name original.ckpt --mode test --save_model False --cuda False

# # on AVIB
# python main.py --infile /ssd1/dchen/GNN_PEPTIDE_TCR/external/ATM-TCR/data/combined_dataset.csv \
# --indepfile /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.avib.alpha_beta_splits.beta.rep_0.ATM-TCR.csv \
# --model_name original.ckpt --mode test --save_model False --cuda False

In [ ]:
# TARPON

# # on NetTCR-2.0
# python fishing.py \
# -b /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.nettcr2.mira_eval_threshold90.TARPON.csv \
# -c /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_tarpon.nettcr2.mira_eval_threshold90.csv

# # on ERGO-II
# python fishing.py \
# -b /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.ergoii.mcpas_test.TARPON.csv \
# -c /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_tarpon.ergoii.mcpas_test.csv

# # on ATM-TCR
# python fishing.py \
# -b /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.atmtcr.combined.TARPON.csv \
# -c /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_tarpon.atmtcr.combined.csv

# # on AVIB
# python fishing.py \
# -b /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test.avib.alpha_beta_splits.beta.rep_0.TARPON.csv \
# -c /ssd1/dchen/GNN_PEPTIDE_TCR/outputs/model_comparison/external_test_tarpon.avib.alpha_beta_splits.beta.rep_0.csv